In [1]:
import pandas as pd

df = pd.read_csv('data/wine-reviews/winemag-data-130k-v2.csv', index_col='wine_id')
df.head(2)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
wine_id,,,,,,,,,,,,,
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos


# **Aggregating Data**

## Groupby
To group information by a certain column we can use the `groupby` method of DataFrames.

What you get back is a `DataFrameGroupBy` object, which you can use in the following ways:
- run one or more aggregation function (e.g. sum, mean, ..), which can be custom made
- loop over it, therefore working on each partial DataFrame for each group

In [2]:
df_by_country = df.groupby('country')

df_by_country

In [3]:
average_points_by_country = df_by_country['points'].mean()

average_points_by_country.head()

country
Argentina                 86.710263
Armenia                   87.500000
Australia                 88.580507
Austria                   90.101345
Bosnia and Herzegovina    86.500000
Name: points, dtype: float64

looping over the `DataFrameGroupBy` allows access to the sub-dataframes containing data for the subgroups alone, which can be useful for complex, non-standard operations.

In [17]:
for group, group_df in df.groupby('country'):
    print(f'The average points for {group:<25} = {group_df["points"].mean():.2f}')

The average points for Argentina                 = 86.71
The average points for Armenia                   = 87.50
The average points for Australia                 = 88.58
The average points for Austria                   = 90.10
The average points for Bosnia and Herzegovina    = 86.50
The average points for Brazil                    = 84.67
The average points for Bulgaria                  = 87.94
The average points for Canada                    = 89.37
The average points for Chile                     = 86.49
The average points for China                     = 89.00
The average points for Croatia                   = 87.22
The average points for Cyprus                    = 87.18
The average points for Czech Republic            = 87.25
The average points for Egypt                     = 84.00
The average points for England                   = 91.58
The average points for France                    = 88.85
The average points for Georgia                   = 87.69
The average points for Germany 

### Multi-column grouping
The grouping can be done over multiple columns, producing a MultiIndexed result

In [6]:
df_by_country_and_variety = df.groupby(['country', 'variety'])   # multi-dimention groupby

count_of_points = df_by_country_and_variety['points'].count()

count_of_points.head()

country    variety                   
Argentina  Barbera                         1
           Bonarda                       105
           Bordeaux-style Red Blend       89
           Bordeaux-style White Blend      1
           Cabernet Blend                  8
Name: points, dtype: int64

To avoid generating a multi-index, the resulting index can be moved to columns instead using the `as_index=False` option

In [20]:
df.groupby(['country', 'variety'], as_index=False)['points'].count()

,country,variety,points
0,Argentina,Barbera,1
1,Argentina,Bonarda,105
2,Argentina,Bordeaux-style Red Blend,89
3,Argentina,Bordeaux-style White Blend,1
4,Argentina,Cabernet Blend,8
5,Argentina,Cabernet Franc,64
6,Argentina,Cabernet Franc-Cabernet Sauvignon,3
7,Argentina,Cabernet Franc-Malbec,4
8,Argentina,Cabernet Sauvignon,540
9,Argentina,Cabernet Sauvignon-Cabernet Franc,1


### Applying a custom function to the groupby object
To apply a custom function to each group we can simply use the `apply` method

In [26]:
def quantile_ratio(x):
    return x.quantile(0.9)/x.quantile(0.5)

df.groupby('country')['points'].apply(quantile_ratio).head()

country
Argentina                 1.045977
Armenia                   1.004571
Australia                 1.033708
Austria                   1.033333
Bosnia and Herzegovina    1.013873
Name: points, dtype: float64

### Performing Multiple Aggregation
In order to perform more than one aggregation we can use the `.agg` method of the `DataFrameGroupBy`

In [23]:
df.groupby('country')['price'].agg(['min', 'mean', 'max']).head()

,min,mean,max
country,,,
Argentina,4.0,24.510117,230.0
Armenia,14.0,14.500000,15.0
Australia,5.0,35.437663,850.0
Austria,7.0,30.762772,1100.0
Bosnia and Herzegovina,12.0,12.500000,13.0


### ***EXERCISE 5.1***
Find the tasters that have:
- reviewed most wine countries
- spent the most money in wine

**HINT**: to count the unique values in a Series you can use the method `nunique`

In [63]:
# insert solution here

### ***EXERCISE 5.2***
Find the most controversial wine variety

In [64]:
# insert solution here

### ***EXERCISE 5.3***
Show the maximum point given by each taster for each region2

In [73]:
# insert solution here

## Pivot Table
The `pivot_table` method of DataFrame allows to create a table by defining:
- what to have as rows (index)
- what to have as columns (columns)
- what column values to put in the table body (values)
- what aggregation to use when multiple values end in the same cell (aggfunc)

index and columns can be lists of columns resulting in MultiIndex

In [81]:
df.pivot_table(index='region_2', columns='points', values='price', aggfunc='mean')

points,80,81,82,83,84,85,86,87,88,89,...,91,92,93,94,95,96,97,98,99,100
region_2,,,,,,,,,,,,,,,,,,,,,
California Other,16.321429,12.222222,12.737327,12.248848,13.138889,13.678756,14.116402,16.073955,18.079832,19.992908,...,29.021739,37.116279,53.052632,56.300000,48.333333,75.000000,75.000000,NaN,NaN,NaN
Central Coast,22.000000,23.514706,25.246862,23.364407,25.166994,26.119632,28.676402,28.469810,31.997717,33.101279,...,40.884547,44.461411,48.696833,55.383292,58.247934,75.727273,82.000000,NaN,NaN,NaN
Central Valley,17.550000,15.105263,16.666667,16.219512,17.095745,18.326733,17.340000,18.993377,19.103175,21.971963,...,22.063492,26.354839,25.666667,27.333333,NaN,NaN,NaN,NaN,NaN,NaN
Columbia Valley,19.000000,16.384615,21.490196,22.229167,20.494048,21.328467,21.767123,24.818731,27.369184,31.256018,...,39.749776,44.203077,50.575676,56.937173,64.184211,79.729730,84.750000,94.285714,125.0,80.0
Finger Lakes,22.800000,17.636364,16.869565,16.666667,19.198020,18.244240,18.976285,19.830645,20.627660,20.862069,...,29.041667,31.952381,35.454545,35.833333,NaN,NaN,NaN,NaN,NaN,NaN
Long Island,18.500000,16.500000,18.916667,14.565217,21.093023,28.000000,25.777778,28.180328,31.990909,29.622222,...,36.923077,47.823529,53.166667,40.000000,NaN,NaN,NaN,NaN,NaN,NaN
Napa,25.333333,31.500000,29.604938,32.265487,35.335714,36.593472,41.406489,41.734577,48.731126,51.040462,...,64.878613,71.388418,83.209924,106.647059,115.912791,141.232143,172.777778,208.214286,202.5,200.0
Napa-Sonoma,30.000000,33.000000,26.200000,24.956522,33.425926,28.815789,33.419048,32.038217,35.419847,37.353846,...,42.821705,54.693069,57.600000,65.282051,72.916667,61.000000,100.000000,NaN,NaN,NaN
New York Other,NaN,11.750000,10.500000,22.307692,18.777778,18.416667,19.250000,23.000000,23.740741,22.058824,...,35.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# **Reshaping Data**
Before or after an aggregation it can be useful to sort data by either values of index.

In [90]:
sorted_by_wine_names = df.sort_values('variety')
sorted_by_wine_names.head(3)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
wine_id,,,,,,,,,,,,,
98235,France,Abouriou is a grape found almost exclusively i...,Just,87,15.0,Southwest France,Côtes du Marmandais,NaN,Roger Voss,@vossroger,Cave du Marmandais 2012 Just Abouriou (Côtes d...,Abouriou,Cave du Marmandais
129894,France,"Despite its proximity to Bordeaux, the Marmand...",NaN,91,15.0,Southwest France,Côtes du Marmandais,NaN,Roger Voss,@vossroger,Lionel Osmin & Cie 2014 Abouriou (Côtes du Mar...,Abouriou,Lionel Osmin & Cie
45401,US,"Comprised 100% of this rare variety, this wine...",Moonlight Sonata,85,75.0,California,Russian River Valley,Sonoma,Virginie Boone,@vboone,Cerridwen 2012 Moonlight Sonata Abouriou (Russ...,Abouriou,Cerridwen


In [91]:
sorted_by_index_again = sorted_by_wine_names.sort_index()
sorted_by_index_again.head(3)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
wine_id,,,,,,,,,,,,,
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm


By default the sorting is done in ascending way, which can be reversed by using the `ascending=False` option.

In [92]:
mean_points_per_country = df.groupby('country')['points'].mean()
mean_points_per_country.sort_values(ascending=False)

country
England                   91.581081
India                     90.222222
Austria                   90.101345
Germany                   89.851732
Canada                    89.369650
Hungary                   89.191781
China                     89.000000
France                    88.845109
Luxembourg                88.666667
Australia                 88.580507
Morocco                   88.571429
Switzerland               88.571429
US                        88.563720
Italy                     88.562231
Israel                    88.471287
New Zealand               88.303030
Portugal                  88.250220
Turkey                    88.088889
Slovenia                  88.068966
South Africa              88.056388
Bulgaria                  87.936170
Georgia                   87.686047
Lebanon                   87.685714
Armenia                   87.500000
Serbia                    87.500000
Spain                     87.288337
Greece                    87.283262
Czech Republic      

### Sorting by multiple columns
sorting by multiple columns can be done by passing a list of the columns on which to sort by

In [94]:
df.sort_values(['points', 'price']).head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
wine_id,,,,,,,,,,,,,
23437,Argentina,Candied plum and red berry aromas smell like r...,Estate Bottled,80,5.0,Mendoza Province,Mendoza,NaN,Michael Schachner,@wineschach,Terrenal 2013 Estate Bottled Malbec (Mendoza),Malbec,Terrenal
125826,Argentina,"While this smells floral and candied, the pala...",NaN,80,5.0,Mendoza Province,Mendoza,NaN,Michael Schachner,@wineschach,Toca Diamonte 2010 Malbec (Mendoza),Malbec,Toca Diamonte
34501,Chile,"Neutral aromas, sour apple and citrus flavors,...",NaN,80,6.0,Curicó Valley,NaN,NaN,Michael Schachner,@wineschach,Quasar 2009 Chardonnay (Curicó Valley),Chardonnay,Quasar
81390,Spain,"Very light, even thin, this white lacks the st...",Mediterranean White,80,6.0,Catalonia,Catalonia,NaN,NaN,NaN,René Barbier NV Mediterranean White White (Cat...,White Blend,René Barbier
99996,Chile,"The nose is sulphuric and a bit barnyardy, whi...",Estate,80,6.0,Maipo Valley,NaN,NaN,Michael Schachner,@wineschach,Santa Alicia 2001 Estate Chardonnay (Maipo Val...,Chardonnay,Santa Alicia


## Stack and Unstack
The `.stack()` and `.unstack()` methods are used to move columns to indecies and viceversa.

This can be handy to convert multi-index Series to more convenient DataFrames. `level` option can be use to select which layer of the MultiIndex to move.

In [96]:
count_by_country_and_variety = df.groupby(['country', 'variety'])['points'].count()
count_by_country_and_variety.head()

country    variety                   
Argentina  Barbera                         1
           Bonarda                       105
           Bordeaux-style Red Blend       89
           Bordeaux-style White Blend      1
           Cabernet Blend                  8
Name: points, dtype: int64

In [101]:
count_by_country_and_variety.unstack(level=0)

country,Argentina,Armenia,Australia,Austria,Bosnia and Herzegovina,Brazil,Bulgaria,Canada,Chile,China,...,Serbia,Slovakia,Slovenia,South Africa,Spain,Switzerland,Turkey,US,Ukraine,Uruguay
variety,,,,,,,,,,,,,,,,,,,,,
Barbera,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,238.0,NaN,NaN
Bonarda,105.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bordeaux-style Red Blend,89.0,NaN,29.0,2.0,NaN,2.0,2.0,15.0,37.0,NaN,...,NaN,NaN,4.0,89.0,9.0,NaN,3.0,1824.0,NaN,NaN
Bordeaux-style White Blend,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,42.0,NaN,NaN
Cabernet Blend,8.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,28.0,1.0,...,NaN,NaN,NaN,2.0,5.0,NaN,NaN,21.0,NaN,NaN
Cabernet Franc,64.0,NaN,NaN,1.0,NaN,1.0,2.0,24.0,38.0,NaN,...,NaN,NaN,1.0,3.0,1.0,NaN,NaN,1003.0,NaN,2.0
Cabernet Franc-Cabernet Sauvignon,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
Cabernet Franc-Malbec,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cabernet Sauvignon,540.0,NaN,266.0,5.0,NaN,4.0,20.0,7.0,771.0,NaN,...,NaN,NaN,3.0,132.0,53.0,NaN,2.0,7315.0,1.0,2.0


### ***EXERCISE 5.4***
Create the equivalent of the following using only `pivot_table` and `stack/unstack`:
```python
df.groupby(['country', 'variety'])['points'].mean()
```

**HINT** To get rid of `NaN` values, add `.dropna()` at the end

In [114]:
# insert solution here